# UC SD Micromasters - Python for Data Science
# Craigslist Used Cars Dataset



In [1]:
# import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Import and read dataset

First, look at downloaded file from https://www.kaggle.com/austinreese/craigslist-carstrucks-data/download.

In [ ]:
from subprocess import check_output
print(check_output(["ls", "./data"]).decode("utf8"))

Unzip the file and extract into the data folder.

In [ ]:
# how to from https://www.kaggle.com/mchirico/how-to-read-datasets
import zipfile
Dataset = "craigslist-carstrucks-data"

with zipfile.ZipFile("./data/"+Dataset+".zip","r") as z:
    z.extractall("./data")

Import data as pandas dataframe.

In [2]:
cars_raw = pd.read_csv("./data/vehicles.csv")

In [ ]:
# dataframe shape
print("Number of rows:",cars_raw.shape[0])
print()
print("Number of columns",cars_raw.shape[1])

## Clean data set

Remove unnecessary columns and null values. Let's look at the proportion of no-null values as percent of total number of rows. This will give us an idea of which variables contain actual data vs those that are mostly `NULL`

In [ ]:
no_null_prc = (cars_raw.count()/len(cars_raw)).round(2)*100
no_null_prc

Select columns of interest that have greater than 80% of actual data.

In [ ]:
cols_to_use = list(no_null_prc[no_null_prc>=80].index)
cols_to_use

Remove the first 4 columns: these columns only contain the id and other url information. Not needed for this analysis.

In [ ]:
cols_not_needed = list(cars_raw.columns[0:4])
cols_not_needed.extend(['description','image_url'])
cols_not_needed

In [3]:
cars = cars_raw[[name for name in cols_to_use if name not in cols_not_needed]]
cars = cars.dropna()
cars.sample(10)

NameError: name 'cols_to_use' is not defined

In [ ]:
# dataframe shape
print("Number of rows:",cars.shape[0])
print()
print("Number of columns",cars.shape[1])

### Description of the variables:

Categorical variables:
- manufacturer
- model
- fuel
- title_status
- transmission
- state

Numerical variables:
- price
- year
- odometer

Location variables:
- lat
- long

Date variables:
- year

## Preliminary data exploration and further cleaning

Firts, lets look at the distribution of the numeric variables : 

- Median price is \$10900 while the max value is $4.3 billion!
- Medin odometer reading is 93827 mi.
- Median year is 2012

Here are some more additiona statistics.

In [ ]:
cars.describe(percentiles=[0.05,0.1,0.5,0.9,0.95]).transpose()

### Price and Odometer

Remove vehicles with prices and odometer readings that are slightly more than the 90% percentile and less than the 1% percentile. This will get rid of extreme outliers. 

In [ ]:
cars = cars[(cars.price<=30000) & (cars.price>1000) & (cars.odometer<=180000) &  (cars.odometer>=2000)]

# dataframe shape
print("Number of rows:",cars.shape[0])
print()
print("Number of columns",cars.shape[1])

Let's look at a plot of a joint histogram using exagonal bins of the price with odometer reading: notice the down-ward non-linear nature of of the data.

In [ ]:
sns.jointplot("odometer", "price", data=cars,size=10,kind = "hex");

### Transmission

A large majority of values in transmission are laballed as  "other":

In [ ]:
cars.transmission.value_counts()

We'll assume transmission labeled "other" is automatic:

In [ ]:
cars['transmission'] = np.where(cars.transmission == "other","automatic",cars.transmission)

Now let's look at the boxplot of transmission type vs price. Notice that vehicles with an automatic transmission tend to be at slightly higher price:

In [ ]:
plt.figure(figsize=(5,5))
sns.boxplot(x='transmission',y='price',data=cars).set_title('Used Cars Dataset: transmission type');

### Title Status

A very small minority of `title_status` values are either labeled "missing" or "parts only". So we'll remove those variables from the dataset.

In [ ]:
cars.title_status.value_counts()

In [ ]:
# remove 'missing' and 'parts only'
keep = ['clean','rebuilt','salvage','lien']
cars = cars[cars['title_status'].isin(keep)]
# dataframe shape
print("Number of rows:",cars.shape[0])
print()
print("Number of columns",cars.shape[1])

Now let's look a the boxplot of title status vs price. Interestingly, vehicles with a lien tend to be higher priced. This could be because those vehicles typically are being sold to pay off creditors (banks, dealerships, etc.)

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x='title_status',y='price',data=cars).set_title('Used Cars Dataset: status title');

### Fuel Type

Let's take a look at the distribution of prices by fuel type:

In [ ]:
plt.figure(figsize=(10,5))
sns.set_style("whitegrid")
sns.boxplot(x='fuel',y='price',data=cars).set_title('Used Cars Dataset: fuel type');

Vehicles with fuel type as  "other" can be removed from the analysis since it would be difficult to identify the fuel to for those:

In [ ]:
cars.fuel.value_counts()

In [ ]:
cars = cars[~cars['fuel'].isin(['other'])]
# dataframe shape
print("Number of rows:",cars.shape[0])
print()
print("Number of columns",cars.shape[1])

### Year

Lets see the number of vehicles grouped by year on a line plot: notice that most vehicles have years between the late 90's and early 2020.

In [ ]:
grouped_by_year = cars.groupby('year').count().reset_index()
plt.figure(figsize=(10,5))
sns.lineplot(x='year',y='price',data=grouped_by_year).set_title('Used Cars Dataset : number of listings for each year');

For this analysis we'll look at listings between 1999 and 2019:

In [ ]:
cars = cars[(cars.year>=1999) & (cars.year<=2019)]
# dataframe shape
print("Number of rows:",cars.shape[0])
print()
print("Number of columns",cars.shape[1])

### Manufacturer and Model Type

Now let's look at the distribution of manufacturer and model type. 

First, we'll look at the counts of model that occurs at least 1000 times in the dataset. 

In [ ]:
model_count = pd.DataFrame(cars['model'].value_counts().reset_index())

model_count = model_count[model_count.model>500] # filter above certain value

print("Total number of unique models that occur at least 500 times:",len(model_count))

In [ ]:
plt.figure(figsize=(10,15))
fig = sns.barplot(x='model',y="index",data=model_count);
fig.set(xlabel="Number of occurences",ylabel="Model",title="Number of Occurrences by Model");

Now we'll look at the number of occurences by manufacturer

In [ ]:
manf_count = pd.DataFrame(cars['manufacturer'].value_counts().reset_index())
print("Total number of occurences for each manufacturer:",len(model_count))

In [ ]:
plt.figure(figsize=(10,10))
fig = sns.barplot(x='manufacturer',y="index",data=manf_count);
fig.set(xlabel="Number of occurences",ylabel="Manufacturer",title="Number of Occurrences by Manufacturer");

Based on these two last plots, we'll remove the values of the dataset that have  `model` occurences of at less than 100 times and `manufacturer` occurences of at least 10 times.

In [ ]:
temp = cars.groupby('model').filter(lambda x: len(x)>100)

In [ ]:
temp

In [ ]:
temp_count = pd.DataFrame(temp['model'].value_counts().reset_index())
temp_count

In [ ]:
plt.figure(figsize=(10,15))
fig = sns.barplot(x='model',y="index",data=temp_count);